In [1]:
%matplotlib inline
import os
from pymongo import MongoClient

If dataset/comments.txt doesn't exist, pull the comments from the database.

In [2]:
if not os.path.isfile('dataset/comments.txt'):
    client = MongoClient()
    scrape = client.scrape

    f = open("dataset/comments.txt", "w")

    # print the comments to a file, separated with \n, 
    # so we can use the croatian stemmer on them
    for comment in scrape.blic.find():
        text = comment['comment']
        text.replace('\n', ' ')
        text += '\n'
        f.write(text.encode('utf8'))

    f.close()

In [33]:
cyrillic = set(u"АаБбВвГгДдЂђЕеЖжЗзИиЈјКкЛлЉљМмНнЊњОоПпРрСсТтЋћУуФфХхЦцЧчЏџШш")
def remove_cyrillic_comments(comments, print_perc=True):
    cyrillic_count = 0.0; all_count = len(comments)
    clean_coms = []
    
    for comment in comments:
        if not bool(set(comment.decode('utf8')).intersection(cyrillic)):
            clean_coms.append(comment)
        else:
            cyrillic_count += 1
        
    if print_perc:
        print "Cyrillic comments make up %s percent" % (cyrillic_count / all_count * 100)
    return clean_coms

corpus = open('dataset/comments.txt', 'r').readlines()[:10**4]
corpus = remove_cyrillic_comments(corpus)



Cyrillic comments make up 1.43 percent


In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

croatian_stop_words = set([u"a",u"ako",u"ali",u"bi",u"bih",u"bila",u"bili",u"bilo",u"bio",u"bismo",u"biste",u"biti",u"bumo",u"da",u"do",u"duž",u"ga",u"hoće",u"hoćemo",u"hoćete",u"hoćeš",u"hoću",u"i",u"iako",u"ih",u"ili",u"iz",u"ja",u"je",u"jedna",u"jedne",u"jedno",u"jer",u"jesam",u"jesi",u"jesmo",u"jest",u"jeste",u"jesu",u"jim",u"joj",u"još",u"ju",u"kada",u"kako",u"kao",u"koja",u"koje",u"koji",u"kojima",u"koju",u"kroz",u"li",u"me",u"mene",u"meni",u"mi",u"mimo",u"moj",u"moja",u"moje",u"mu",u"na",u"nad",u"nakon",u"nam",u"nama",u"nas",u"naš",u"naša",u"naše",u"našeg",u"ne",u"nego",u"neka",u"neki",u"nekog",u"neku",u"nema",u"netko",u"neće",u"nećemo",u"nećete",u"nećeš",u"neću",u"nešto",u"ni",u"nije",u"nikoga",u"nikoje",u"nikoju",u"nisam",u"nisi",u"nismo",u"niste",u"nisu",u"njega",u"njegov",u"njegova",u"njegovo",u"njemu",u"njezin",u"njezina",u"njezino",u"njih",u"njihov",u"njihova",u"njihovo",u"njim",u"njima",u"njoj",u"nju",u"no",u"o",u"od",u"odmah",u"on",u"ona",u"oni",u"ono",u"ova",u"pa",u"pak",u"po",u"pod",u"pored",u"prije",u"s",u"sa",u"sam",u"samo",u"se",u"sebe",u"sebi",u"si",u"smo",u"ste",u"su",u"sve",u"svi",u"svog",u"svoj",u"svoja",u"svoje",u"svom",u"ta",u"tada",u"taj",u"tako",u"te",u"tebe",u"tebi",u"ti",u"to",u"toj",u"tome",u"tu",u"tvoj",u"tvoja",u"tvoje",u"u",u"uz",u"vam",u"vama",u"vas",u"vaš",u"vaša",u"vaše",u"već",u"vi",u"vrlo",u"za",u"zar",u"će",u"ćemo",u"ćete",u"ćeš",u"ću",u"što"])

vectorizer = TfidfVectorizer(
    strip_accents="unicode",
    lowercase=True,
    ngram_range=(1,2),
    min_df=5,
    norm='l2',
    smooth_idf=True,
    use_idf=True,
    stop_words=croatian_stop_words)

vectorizer.fit_transform(corpus)



<100000x34414 sparse matrix of type '<type 'numpy.float64'>'
	with 1004646 stored elements in Compressed Sparse Row format>

In [22]:
import numpy as np
popularity_ind = np.argsort(vectorizer.vocabulary_.values())